In [1]:
import json
from pathlib import Path
from multiprocessing import Pool

import pandas as pd

In [2]:
import sys
sys.path.append('../src')

from utils import run_cmd

In [3]:
def run_ebeis(infile, outfile, prefix):
    program = "/home/chen1i6c04/Tools/EToKi/EToKi.py"
    run_cmd(f"{program} EBEis -q {infile} -p {prefix} > {outfile}")

In [5]:
dirpath = Path('/media/NGS/Data_Analysis/20210401_NHRI_Ecoli/Contigs')
outpath = Path('/media/NGS/Data_Analysis/20210401_NHRI_Ecoli/Analysis')

In [6]:
program = "/home/chen1i6c04/Tools/EToKi/EToKi.py"
cmds = []
for filepath in dirpath.iterdir():
    outfile = outpath/filepath.stem/('EBEis.json')
    cmd = f"{program} EBEis -q {filepath} -p {filepath.stem} > {outfile}"
    cmds.append(cmd)

In [8]:
with Pool(64) as p:
    p.map(run_cmd, cmds)
    p.close()
    p.join()

In [6]:
def parse_ebeis_output(filepath):
    with open(filepath) as handle:
        data = json.loads(handle.read())
    return data

In [7]:
dirpath = Path('/media/NGS/Data_Analysis/20210401/EBEis')

In [8]:
data = [parse_ebeis_output(filepath) for filepath in dirpath.iterdir()]

df = pd.DataFrame(data)
df = df.drop('query', axis=1)
df = df.set_index('prefix')

In [9]:
df.to_csv('/media/NGS/Data_Analysis/20210401/EBEis.tsv', sep='\t')